# Prelimaries & Setup


In [ ]:
import ast
import itertools as it
import os
import random
import tempfile

from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter as mpl_FuncFormatter
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
from teeplot import teeplot as tp
from tqdm import tqdm
from scipy import stats as scipy_stats
from statsmodels import stats as statsmodels_stats
import seaborn as sns


In [ ]:
random.seed(1)
np.random.seed(1)


In [ ]:
nbm.print_metadata()


# Fetch Data


In [ ]:
df = pd.read_csv("https://osf.io/45b6h/download", compression="gzip")
df


In [ ]:
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))


In [ ]:
# strip out implementation-detail -opt suffixes
df["EVAL_FIT_EST_MODE"] = df["EVAL_FIT_EST_MODE"].str.replace("-opt", "")


In [ ]:
df["EVAL_FIT_EST_MODE-EVAL_MODE"] = (
    df["EVAL_FIT_EST_MODE"]
    + "-"
    + df["EVAL_MODE"]
)


In [ ]:
# subset data to keep exploratory analyses tractable
# df = df[(df["update"] == 20000) & (df["TEST_DOWN_SAMPLE_RATE"] == 0.5)]
df = df[(df["update"] == 20000)]
df = df.sample(frac=0.1)


# Convert Data Vectors to Long-Form

i.e., each trait of each individual is a single row


In [ ]:
vector_columns = (
    #     "genome",
    "phenotype",
    "traits_attempted_estimations",
    "traits_estimation_dist",
    "traits_estimated_scores",
    "traits_estimation_source_ids",
    "traits_evaluated",
    "traits_successful_estimations",
)


In [ ]:
for column in tqdm(vector_columns):
    print(column)
    df[column] = df[column].apply(ast.literal_eval)


In [ ]:
df["vector len"] = df["traits_evaluated"].apply(len)
for column in tqdm(vector_columns):
    print(column)
    assert (df[column].apply(len) == df["vector len"]).all()


In [ ]:
df["vector index"] = df["vector len"].apply(lambda x: [*range(x)])


In [ ]:
df["num traits evaluated"] = df["traits_evaluated"].apply(sum)
df["num attempted trait estimations"] = df["traits_attempted_estimations"].apply(sum)
df["num successful trait estimations"] = df["traits_successful_estimations"].apply(sum)


In [ ]:
(df["num successful trait estimations"] == df["num attempted trait estimations"]).all()


In [ ]:
# chunk to prevent running out of memory
chunk_size = 32768
num_chunks = (len(df) + chunk_size - 1) // chunk_size

exploded_chunks = []
for i in tqdm(range(num_chunks)):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    chunk = df.iloc[start_idx:end_idx]

    exploded_chunk = chunk.explode(["vector index", *vector_columns])
    exploded_chunks.append(exploded_chunk)


In [ ]:
# concatenate chunks to single csv, delete from memory, then reload from csv
with tempfile.NamedTemporaryFile(mode="w") as tmpfile:
    for i, exploded_chunk in enumerate(tqdm(exploded_chunks)):
        kwargs = {"index": False, "chunksize": 4096}
        if i:
            kwargs["mode"] = "a"
            kwargs["header"] = False
        exploded_chunk.to_csv(tmpfile.name, **kwargs)
        tmpfile.flush()

    del exploded_chunks
    exploded_df = pd.read_csv(tmpfile.name)

exploded_df


# Setup New Columns needed for Analyses


In [ ]:
len(exploded_df)


## Raw Trait Error


In [ ]:
exploded_df["trait estimation error"] = (
    exploded_df["traits_estimated_scores"]
    - exploded_df["phenotype"]
)
exploded_df["trait estimation abs error"] = exploded_df["trait estimation error"].abs()


## Normalized Trait Values


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "vector index",
    "update",
]
exploded_df["normalized phenotype"] = exploded_df.groupby(
    groupby_columns,
)["phenotype"].rank(pct=True)


def add_normalized_trait_estimate(df: pd.DataFrame) -> pd.DataFrame:
    df["normalized trait estimate"] = scipy_stats.percentileofscore(
        a=df["phenotype"],
        score=df["traits_estimated_scores"],
    ) / 100
    return df
exploded_df = exploded_df.groupby(
    groupby_columns,
    group_keys=False,
).apply(
    add_normalized_trait_estimate,
)


In [ ]:
exploded_df["normalized trait estimation error"] = (
    exploded_df["normalized trait estimate"]
    - exploded_df["normalized phenotype"]
)
exploded_df["abs normalized trait estimation error"] = (
    exploded_df["normalized trait estimation error"].abs()
)


## Normalized Trait Values Error %


In [ ]:
exploded_df["normalized trait estimation error %"] = (
    exploded_df["normalized trait estimation error"] * 100
)
exploded_df["abs normalized trait estimation error %"] = (
    exploded_df["abs normalized trait estimation error"] * 100
)


## Trait Estimation Outcome


In [ ]:
exploded_df["is evaluated"] = exploded_df["traits_evaluated"] == 1
exploded_df["is evaluated"].sum()


In [ ]:
exploded_df["is attempted estimation"] = (
    exploded_df["traits_attempted_estimations"] == 1
)
exploded_df["is attempted estimation"].sum()


In [ ]:
exploded_df["is successful estimation"] = (
    exploded_df["traits_successful_estimations"] == 1
)
exploded_df["is successful estimation"].sum()


In [ ]:
exploded_df["is failed estimation"] = (
    exploded_df["is attempted estimation"]
    & ~exploded_df["is successful estimation"]
)
exploded_df["is failed estimation"].sum()


In [ ]:
assert not (
    exploded_df["is successful estimation"] & exploded_df["is failed estimation"]
).any()


In [ ]:
exploded_df["estimation outcome"] = "na"
exploded_df.loc[
    exploded_df["is failed estimation"],
    "estimation outcome",
] = "failed"
exploded_df.loc[
    exploded_df["is successful estimation"],
    "estimation outcome",
] = "successful"


In [ ]:
exploded_df["is evaluated and is attempted estimation"] = (
    exploded_df["is attempted estimation"] & exploded_df["is evaluated"]
)


In [ ]:
exploded_df["is neither evaluated nor attempted estimation"] = (
    (~exploded_df["is attempted estimation"]) & (~exploded_df["is evaluated"])
)


# Plot: estimation mode vs estimation failure/attempts


In [ ]:
def catplot_bar_label(*args, **kwargs):
    g = sns.catplot(
        *args,
        **kwargs,
        margin_titles=True,
    )
    for ax in g.axes.flat:
        for container in ax.containers:
            ax.bar_label(container, label_type='edge')

tp.tee(
    catplot_bar_label,
    data=exploded_df,
    col="EVAL_FIT_EST_MODE-EVAL_MODE",
    row="TEST_DOWN_SAMPLE_RATE",
    x="DIAGNOSTIC",
    hue="estimation outcome",
    kind="count",
)


In [ ]:
tp.tee(
    catplot_bar_label,
    data=exploded_df,
    col="EVAL_FIT_EST_MODE-EVAL_MODE",
    row="DIAGNOSTIC",
    x="TEST_DOWN_SAMPLE_RATE",
    y="is evaluated",
    kind="bar",
)


In [ ]:
tp.tee(
    catplot_bar_label,
    data=exploded_df,
    col="EVAL_FIT_EST_MODE-EVAL_MODE",
    row="DIAGNOSTIC",
    x="TEST_DOWN_SAMPLE_RATE",
    y="is evaluated and is attempted estimation",
    kind="bar",
)


In [ ]:
tp.tee(
    catplot_bar_label,
    data=exploded_df,
    col="EVAL_FIT_EST_MODE-EVAL_MODE",
    row="DIAGNOSTIC",
    x="TEST_DOWN_SAMPLE_RATE",
    y="is neither evaluated nor attempted estimation",
    kind="bar",
)


In [ ]:
tp.tee(
    catplot_bar_label,
    data=exploded_df,
    col="EVAL_FIT_EST_MODE-EVAL_MODE",
    row="DIAGNOSTIC",
    x="TEST_DOWN_SAMPLE_RATE",
    y="is attempted estimation",
    kind="bar",
)


# Plot: estimated score vs phenotype

by diagnostic and estimate mode


In [ ]:
def facet_scatterplot(data, col, row, x, y):
    g = sns.FacetGrid(
        data=data,
        col=col,
        row=row,
        margin_titles=True,
    )
    g.map(
        sns.scatterplot,
        x,
        y,
    )

tp.tee(
    facet_scatterplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
    ],
    col="DIAGNOSTIC",
    row="EVAL_FIT_EST_MODE-EVAL_MODE",
    x="phenotype",
    y="traits_estimated_scores",
)


# Plot: trait estimation error vs phylogenetic distance

for each diagnostic and estimation mode


In [ ]:
def facet_regplot(data, x, y, col, row):
    g = sns.FacetGrid(
        data=data,
        col=col,
        row=row,
        margin_titles=True,
        sharex=False,
        sharey="row",
    )
    g.map(
        sns.regplot,
        x,
        y,
        n_boot=10,
        scatter_kws={
            "color": "red",
            "alpha": 0.1,
        },
    ).set(
        xscale="log",
        yscale="log",
    )

tp.tee(
    facet_regplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    col="DIAGNOSTIC",
    row="EVAL_FIT_EST_MODE-EVAL_MODE",
    x="traits_estimation_dist",
    y="abs normalized trait estimation error",
)


In [ ]:
def symlog(x, linthresh=1):
    if linthresh <= 0:
        raise ValueError("linthresh should be positive")

    x_abs = np.abs(x)
    sign = np.sign(x)

    log_part = np.log10(x_abs - linthresh + 1) * (x_abs > linthresh)
    lin_part = x * (x_abs <= linthresh)

    return sign * (log_part + lin_part)


In [ ]:
def symlog_formatter(x, pos):
    sign = '-' if x < 0 else ''
    x_abs = abs(x)
    return f'{sign}$10^{{{x_abs}}}$' if x else "0"

def facet_histplot(data, x, y, hue, col, row):
    g = sns.FacetGrid(
        data=data,
        col=col,
        row=row,
        margin_titles=True,
        sharex=True,
        sharey="row",
        aspect=1,
    )
    g.map(
        sns.histplot,
        x,
        y,
        hue,
        binwidth=(1, 0.1),
        palette=sns.color_palette(["blue"]),
        common_norm=False,
    ).set_axis_labels(
        x_var="Log2 Estimation Distance",
        y_var="Trait Estimate Error,\nNormalized %",
    ).set_titles(
        col_template="{col_var}\n{col_name}",
        row_template="MODE\n{row_name}",
    )

    # Create a FuncFormatter instance with the custom function
    formatter = mpl_FuncFormatter(symlog_formatter)
    for ax in g.axes.flat:
        ax.yaxis.set_major_formatter(formatter)

    plt.tight_layout()



# tp.tee(
#     sns.displot,
exploded_df["log2 traits_estimation_dist"] = np.log2(exploded_df["traits_estimation_dist"])
exploded_df["symlog normalized trait estimation error %"] = symlog(exploded_df["normalized trait estimation error %"])

tp.tee(
    facet_histplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    col="DIAGNOSTIC",
    row="EVAL_FIT_EST_MODE-EVAL_MODE",
    x="log2 traits_estimation_dist",
    y="symlog normalized trait estimation error %",
    hue="log2 traits_estimation_dist",
)


In [ ]:
def facet_histplot(data, x, y, hue, col, row):
    g = sns.FacetGrid(
        data=data,
        col=col,
        row=row,
        margin_titles=True,
        sharex=True,
        sharey="row",
        aspect=1,
    )
    g.map(
        sns.histplot,
        x,
        y,
        hue,
        binwidth=(1, 0.1),
        palette=sns.color_palette(["blue"]),
        common_norm=False,
    ).set_axis_labels(
        x_var="Log2 Estimation Distance",
        y_var="Trait Estimate Error,\nAbs Normalized %",
    ).set_titles(
        col_template="{col_var}\n{col_name}",
        row_template="MODE\n{row_name}",
    )

    # Create a FuncFormatter instance with the custom function
    formatter = mpl_FuncFormatter(symlog_formatter)
    for ax in g.axes.flat:
        ax.yaxis.set_major_formatter(formatter)

    plt.tight_layout()



exploded_df["log2 traits_estimation_dist"] = np.log2(exploded_df["traits_estimation_dist"])
exploded_df["symlog abs normalized trait estimation error %"] = symlog(exploded_df["abs normalized trait estimation error %"])
tp.tee(
    facet_histplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    col="DIAGNOSTIC",
    row="EVAL_FIT_EST_MODE-EVAL_MODE",
    x="log2 traits_estimation_dist",
    y="symlog abs normalized trait estimation error %",
    hue="log2 traits_estimation_dist",
)


# Plot: mean estimation error by diagnostic/estimation mode


In [ ]:
def facet_barplot(data, x, y, row, col):
    g = sns.FacetGrid(
        data=data,
        row=row,
        col=col,
        margin_titles=True,
        sharey="row",
        sharex=False,
    )
    g.map(
        sns.barplot,
        x,
        y,
    )

tp.tee(
    facet_barplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    x="EVAL_FIT_EST_MODE-EVAL_MODE",
    y="trait estimation abs error",
    row="DIAGNOSTIC",
    col="TEST_DOWN_SAMPLE_RATE",
)


## normalized


In [ ]:
facet_barplot(
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    x="EVAL_FIT_EST_MODE",
    y="abs normalized trait estimation error",
    col="DIAGNOSTIC",
    row="TEST_DOWN_SAMPLE_RATE",
)


In [ ]:
facet_barplot(
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    x="EVAL_FIT_EST_MODE",
    y="abs normalized trait estimation error",
    col="DIAGNOSTIC",
    row="TEST_DOWN_SAMPLE_RATE",
)


# normalized, not absolute value


In [ ]:
facet_barplot(
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    x="EVAL_FIT_EST_MODE",
    y="normalized trait estimation error",
    col="DIAGNOSTIC",
    row="TEST_DOWN_SAMPLE_RATE",
)


# Plot: phylogenetic estimation distance by diagnostic/estimation mode


In [ ]:
tp.tee(
    facet_barplot,
    data=exploded_df[
        exploded_df["is successful estimation"]
        & (exploded_df["traits_estimation_dist"] > 0)
    ],
    row="DIAGNOSTIC",
    col="TEST_DOWN_SAMPLE_RATE",
    x="EVAL_FIT_EST_MODE",
    y="traits_estimation_dist",
)


# Plot: error distributions

by diagnostic/evaluation mode


In [ ]:
def facet_violinplot(data, x, y, col, row):
    g = sns.FacetGrid(
        data=data,
        col=col,
        row=row,
        margin_titles=True,
        sharex=False,
    )
    g.map(
        sns.violinplot,
        x,
        y,
    )

tp.tee(
    facet_violinplot,
    data=exploded_df[exploded_df["is successful estimation"]],
    x="trait estimation abs error",
    y="EVAL_FIT_EST_MODE-EVAL_MODE",
    col="DIAGNOSTIC",
    row="TEST_DOWN_SAMPLE_RATE",
)


In [ ]:
tp.tee(
    facet_violinplot,
    data=exploded_df[exploded_df["is successful estimation"]],
    x="abs normalized trait estimation error",
    y="EVAL_FIT_EST_MODE",
    col="DIAGNOSTIC",
    row="TEST_DOWN_SAMPLE_RATE",
)


# Statistics


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "update",
]
records = []
for group, group_df in exploded_df.groupby(groupby_columns):
    group_attrs = dict(zip(groupby_columns, group))
    res = scipy_stats.spearmanr(
        group_df["trait estimation abs error"],
        group_df["traits_estimation_dist"],
    )
    records.append(
        {
            **group_attrs,
            **{
                "error vs. dist spearmanr correlation statistic" : res.statistic,
                "error vs. dist spearmanr correlation p" : res.pvalue,
            },
        }
    )

pd.DataFrame.from_records(records)


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "update",
]
records = []
for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    group_attrs = dict(zip(groupby_columns, group))
    data = group_df["normalized trait estimation error"]
    records.append(
        {
            **group_attrs,
            **{
                "mean normalized error" : data.mean(),
                "median normalized error" : data.median(),
                "1st percentile normalized error" : scipy_stats.scoreatpercentile(data, 1),
                "5th percentile normalized error" : scipy_stats.scoreatpercentile(data, 5),
                "95th percentile normalized error" : scipy_stats.scoreatpercentile(data, 95),
                "99th percentile normalized error" : scipy_stats.scoreatpercentile(data, 99),
            },
#             **dict(
#                 zip(
#                     ("95% CI lower bound", "95% CI upper bound"),
#                     scipy_stats.bootstrap(
#                         (data,),
#                         np.mean,
#                         batch=9,
#                         n_resamples=9,
#                     ).confidence_interval,
#                 )
#             ),
#             **dict(
#                 zip(
#                     ("99% CI lower bound", "99% CI upper bound"),
#                     scipy_stats.bootstrap(
#                         (data,),
#                         np.mean,
#                         batch=9,
#                         n_resamples=9,
#                         confidence_level=0.99,
#                     ).confidence_interval,
#                 )
#             )
        }
    )

pd.DataFrame.from_records(records)


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "update",
]
records = []
for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    group_attrs = dict(zip(groupby_columns, group))
    data = group_df["abs normalized trait estimation error"]
    records.append(
        {
            **group_attrs,
            **{
                "mean normalized absolute error" : data.mean(),
                "median normalized absolute error" : data.median(),
                "1st percentile normalized absolute error" : scipy_stats.scoreatpercentile(data, 1),
                "5th percentile normalized absolute error" : scipy_stats.scoreatpercentile(data, 5),
                "95th percentile normalized absolute error" : scipy_stats.scoreatpercentile(data, 95),
                "99th percentile normalized absolute error" : scipy_stats.scoreatpercentile(data, 99),
            },
        }
    )

pd.DataFrame.from_records(records)


In [ ]:
exploded_df["is failed estimation"].any()


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "EVAL_FIT_EST_MODE",
    "update",
]
records = []
for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    group_attrs = dict(zip(groupby_columns, group))
    stat, p_value = scipy_stats.kruskal(
        *dict([*group_df.groupby("DIAGNOSTIC")["abs normalized trait estimation error"]]).values()
    )
    records.append(
        {
            **group_attrs,
            **{
                "normalized error by diagnostic kruskal-wallis p value" : p_value,
                "normalized error by diagnostic kruskal-wallis statistic" : stat,
            },
        }
    )

pd.DataFrame.from_records(records)


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "TEST_DOWN_SAMPLE_RATE",
    "EVAL_FIT_EST_MODE",
    "update",
]

records = []

for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    group_attrs = dict(zip(groupby_columns, group))
    diagnostic_groups = dict([*group_df.groupby("DIAGNOSTIC")["abs normalized trait estimation error"]])

    pairwise_comparisons = [*it.combinations(diagnostic_groups.keys(), 2)]
    p_values = []
    for first_diagnostic, second_diagnostic in pairwise_comparisons:
        stat, p_value = scipy_stats.mannwhitneyu(
            diagnostic_groups[first_diagnostic],
            diagnostic_groups[second_diagnostic],
            alternative='two-sided',
        )
        p_values.append(p_value)

    # Apply Bonferroni correction
    corrected_p_values = statsmodels_stats.multitest.multipletests(
        p_values, method='bonferroni'
    )[1]

    for (
        (first_diagnostic, second_diagnostic),
        p_value,
        corrected_p_value,
    ) in zip(pairwise_comparisons, p_values, corrected_p_values):
        if np.median(diagnostic_groups[first_diagnostic]) < np.median(diagnostic_groups[second_diagnostic]):
            lower_error_diagnostic = first_diagnostic
            higher_error_diagnostic = second_diagnostic
        else:
            lower_error_diagnostic = second_diagnostic
            higher_error_diagnostic = first_diagnostic

        records.append(
            {
                **group_attrs,
                "lower error diagnostic": lower_error_diagnostic,
                "higher error diagnostic": higher_error_diagnostic,
                "p value": p_value,
                "corrected p value": corrected_p_value,
            }
        )

pd.DataFrame.from_records(records)


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "update",
]
records = []
for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    try:
        group_attrs = dict(zip(groupby_columns, group))
        stat, p_value = scipy_stats.kruskal(
            *dict([*group_df.groupby("TEST_DOWN_SAMPLE_RATE")["abs normalized trait estimation error"]]).values()
        )
        records.append(
            {
                **group_attrs,
                **{
                    "normalized error by diagnostic kruskal-wallis p value" : p_value,
                    "normalized error by diagnostic kruskal-wallis statistic" : stat,
                },
            }
        )
    except ValueError:
        # due to apparent bug in ancestor 0.2 fit est mode name
        continue

pd.DataFrame.from_records(records)


In [ ]:
groupby_columns = [
    "EVAL_MODE",
    "DIAGNOSTIC",
    "EVAL_FIT_EST_MODE",
    "update",
]

records = []

for group, group_df in tqdm(exploded_df.groupby(groupby_columns)):
    try:
        group_attrs = dict(zip(groupby_columns, group))
        downsample_groups = dict([*group_df.groupby("TEST_DOWN_SAMPLE_RATE")["abs normalized trait estimation error"]])

        pairwise_comparisons = [*it.combinations(downsample_groups.keys(), 2)]
        p_values = []
        for first_downsample, second_downsample in pairwise_comparisons:
            stat, p_value = scipy_stats.mannwhitneyu(
                downsample_groups[first_downsample],
                downsample_groups[second_downsample],
                alternative='two-sided',
            )
            p_values.append(p_value)

        # Apply Bonferroni correction
        corrected_p_values = statsmodels_stats.multitest.multipletests(
            p_values, method='bonferroni',
        )[1]

        for (
            (first_downsample, second_downsample),
            p_value,
            corrected_p_value,
        ) in zip(pairwise_comparisons, p_values, corrected_p_values):
            if np.median(downsample_groups[first_downsample]) < np.median(downsample_groups[second_downsample]):
                lower_error_downsample = first_downsample
                higher_error_downsample = second_downsample
            else:
                lower_error_downsample = second_downsample
                higher_error_downsample = first_downsample

            records.append(
                {
                    **group_attrs,
                    "lower error downsample": lower_error_downsample,
                    "higher error downsample": higher_error_downsample,
                    "p value": p_value,
                    "corrected p value": corrected_p_value,
                }
            )
    except (ValueError, ZeroDivisionError):
        # due to apparent bug in ancestor 0.2 fit est mode name
        pass

pd.DataFrame.from_records(records)
